In [124]:
import numpy as np
import time

In [125]:
def diff_matrix_2d(Mx, My, mat1, mat2):
    diff = np.zeros((My,Mx), dtype=complex)
    for i in range(My):
        for j in range(Mx):
            diff[i,j] = np.abs(mat2[i,j]-mat1[i,j])

    avgdiff = np.average(diff)
    stddiff = np.std(diff)
    maxdiff = np.max(diff)
    mindiff = np.min(diff)

    print("Max value 1 =", np.max(mat1))
    print("Max value 2 =", np.max(mat2), '\n')

    print("Min value 1 =", np.min(mat1))
    print("Min value 2 =", np.min(mat2), '\n')

    print("Avg diff =", avgdiff)
    print("Std diff =", stddiff)
    print("Max diff =", maxdiff)
    print("Min diff =", mindiff, '\n')

    print("Mat 1 [3,4] =", mat1[3,4])
    print("Mat 2 [3,4] =", mat2[3,4])
    print("Ex. diff [3,4] =", diff[3,4])

In [126]:
def diff_matrix_3d(Mx, My, n, mat1, mat2):
    diff = np.zeros((My,Mx,n), dtype=complex)
    for i in range(My):
        for j in range(Mx):
            for m in range(n):
                diff[i,j,m] = np.abs(mat2[i,j,m]-mat1[i,j,m])

    avgdiff = np.average(diff)
    stddiff = np.std(diff)
    maxdiff = np.max(diff)
    mindiff = np.min(diff)

    print("Max value mat 1 =", np.max(mat1))
    print("Max value mat 2 =", np.max(mat2), '\n')

    print("Min value mat 1 =", np.min(mat1))
    print("Min value mat 2 =", np.min(mat2), '\n')

    print("Avg diff =", avgdiff)
    print("Std diff =", stddiff)
    print("Max diff =", maxdiff)
    print("Min diff =", mindiff, '\n')

    print("Mat 1 [3,4,5] =", mat1[3,4,5])
    print("Mat 2 [3,4,5] =", mat2[3,4,5])
    print("Ex. diff [3,4,5] =", diff[3,4,5])

Sample parameters

In [127]:
n = 256

My = 10
Mx = 10

tx = 90
ty = 100

qx = 0.0
qy = 0.0 

B = 1.0

psi = np.random.rand(My,Mx,n)

### Rotor kinetic energy

In [139]:
k2  = -np.append(np.arange(0,n/2+1),np.arange(-n/2+1,0))**2 # make second derivative matrix

'''
Method new
'''
H_rot1 = -B*np.fft.ifft(k2*np.fft.fft(psi)).astype(complex)

'''
Method old
'''
H_rot2 = np.zeros((My,Mx,n), dtype=complex)
for i in range(My):
    for j in range(Mx):
        H_rot2[i,j] = -B*np.fft.ifft(k2*np.fft.fft(psi[i,j]))

Hrot_diff = H_rot1-H_rot2

maxdiff = np.max(Hrot_diff)
mindiff = np.min(Hrot_diff)

print("Shape of H_psi method 1 =", H_rot1.shape)
print("Shape of H_psi method 2 =", H_rot2.shape, '\n')

print("max diff of matrix elements", maxdiff)
print("min diff of matrix elements", mindiff, '\n')

diff_matrix_3d(Mx, My, n, H_rot1, H_rot2)

Shape of H_psi method 1 = (10, 10, 256)
Shape of H_psi method 2 = (10, 10, 256) 

max diff of matrix elements 0j
min diff of matrix elements 0j 

Max value mat 1 = (5896.171666165636-3.410605131648481e-13j)
Max value mat 2 = (5896.171666165636-3.410605131648481e-13j) 

Min value mat 1 = (-5971.884025303724-1.7053025658242404e-13j)
Min value mat 2 = (-5971.884025303724-1.7053025658242404e-13j) 

Avg diff = 0j
Std diff = 0.0
Max diff = 0j
Min diff = 0j 

Mat 1 [3,4,5] = (-1672.3182192636987-8.100187187665142e-13j)
Mat 2 [3,4,5] = (-1672.3182192636987-8.100187187665142e-13j)
Ex. diff [3,4,5] = 0j


### Electron kinetic energy

In [129]:
def compute_T_matrix_eom(Tarr):
    '''
        Computes: the transfer integrals in the equations of motion

        ----
        Inputs:
            Tarr (2-dimensional: (My,Mx)): contains the overlaps of the single rotor wavefunctions
        ----

        ----
        Outputs:
            Tarr_new (3-dimensional: (My,Mx,n)): the muliplied transfer integrals except the (i,j) one
        ----
    '''
    T_cur_arr = Tarr.copy()
    Tarr_new = np.zeros(Tarr.shape, dtype=complex)

    for i in range(My):
        for j in range(Mx):
            T_cur_arr[i,j] = 1.0+0j
            Tarr_new[i,j] = np.prod(T_cur_arr)

            T_cur_arr[i,j] = Tarr[i,j]

    Tarr_new = Tarr_new[:, :, np.newaxis]
    return Tarr_new

In [130]:
def get_next_y_rotor(psi):
    return np.roll(psi, -1, axis=0)
    
def get_prev_y_rotor(psi):
    return np.roll(psi, 1, axis=0)
    
def get_next_x_rotor(psi):
    return np.roll(psi, -1, axis=1)
    
def get_prev_x_rotor(psi):
    return np.roll(psi, 1, axis=1)

Method 1 - now used, faster

In [131]:
tic = time.time()

psi_conj = np.conjugate(psi)

'''
arrays of transfer integrals: all are (My,Mx) objects
'''
TD_arr1 = np.einsum('ijk,ijk->ij', psi_conj, get_next_y_rotor(psi), dtype=complex)
TU_arr1 = np.einsum('ijk,ijk->ij', psi_conj, get_prev_y_rotor(psi), dtype=complex)
TR_arr1 = np.einsum('ijk,ijk->ij', psi_conj, get_next_x_rotor(psi), dtype=complex)
TL_arr1 = np.einsum('ijk,ijk->ij', psi_conj, get_prev_x_rotor(psi), dtype=complex)

'''
products of transfer integrals: in every entry (i,j), the (i,j)-th produc is missing
'''
TDr1 = compute_T_matrix_eom(TD_arr1)
TUr1 = compute_T_matrix_eom(TU_arr1)
TRr1 = compute_T_matrix_eom(TR_arr1)
TLr1 = compute_T_matrix_eom(TL_arr1)

'''
electron kinetic energy
'''
H_ele1 = np.zeros((My,Mx,n), dtype=complex)
H_ele1 -= ty*( \
            np.exp(-1j*(2*np.pi*qy/My))*TDr1*get_next_y_rotor(psi) \
            + np.exp(+1j*(2*np.pi*qy/My))*TUr1*get_prev_y_rotor(psi)) 
H_ele1 -= tx*( \
            np.exp(-1j*(2*np.pi*qx/Mx))*TRr1*get_next_x_rotor(psi) \
            + np.exp(+1j*(2*np.pi*qx/Mx))*TLr1*get_prev_x_rotor(psi))

toc = time.time()
print("\nExecution time =", toc-tic)


Execution time = 0.003786802291870117


Method 2 - used before, with loops

In [132]:
tic = time.time()

TL_arr2 = np.zeros((My,Mx), dtype=complex)
TR_arr2 = np.zeros((My,Mx), dtype=complex)
TU_arr2 = np.zeros((My,Mx), dtype=complex)
TD_arr2 = np.zeros((My,Mx), dtype=complex)

# compute arrays for all the pairwise (column/row wise) overlaps
for k in range(My):
    for p in range(Mx):
        TD_arr2[k,p] = np.sum(np.conjugate(psi[k,p])*psi[(k+1)%My,p])
        TU_arr2[k,p] = np.sum(np.conjugate(psi[k,p])*psi[k-1,p])
            
        TR_arr2[k,p] = np.sum(np.conjugate(psi[k,p])*psi[k,(p+1)%Mx])
        TL_arr2[k,p] = np.sum(np.conjugate(psi[k,p])*psi[k,p-1])

TD2 = np.prod(TD_arr2)
TU2 = np.prod(TU_arr2)
TR2 = np.prod(TR_arr2)
TL2 = np.prod(TL_arr2)

TLr2 = np.zeros((My,Mx), dtype=complex)
TRr2 = np.zeros((My,Mx), dtype=complex)
TUr2 = np.zeros((My,Mx), dtype=complex)
TDr2 = np.zeros((My,Mx), dtype=complex)
for i in range(My):
    for j in range(Mx):
        TDr2[i, j] = TD2 / TD_arr2[i, j]
        TUr2[i, j] = TU2 / TU_arr2[i, j]
        TRr2[i, j] = TR2 / TR_arr2[i, j]
        TLr2[i, j] = TL2 / TL_arr2[i, j]

H_ele2 = np.zeros((My,Mx,n), dtype=complex)
for i in range(My):
    for j in range(Mx):
        # for every (i,j) rotor wave function we need to add the calculated transfer integrals
        H_ele2[i,j] += - ty*( \
            np.exp(-1j*(2*np.pi*qy/My))*TDr2[i, j]*psi[(i+1)%My,j] \
            + np.exp(+1j*(2*np.pi*qy/My))*TUr2[i, j]*psi[i-1,j])
        H_ele2[i,j] += - tx*( \
            np.exp(-1j*(2*np.pi*qx/Mx))*TRr2[i, j]*psi[i,(j+1)%Mx] \
            + np.exp(+1j*(2*np.pi*qx/Mx))*TLr2[i, j]*psi[i,j-1])
        
toc = time.time()
print("\nExecution time =", toc-tic)


Execution time = 0.005485057830810547


In [133]:
print("Difference of Transfer Integrals: \n")
print("TLr:\n")
diff_matrix_2d(Mx,My, TLr1, TLr2)

Difference of Transfer Integrals: 

TLr:

Max value 1 = (6.2110682292959916e+178+0j)
Max value 2 = (6.211068229295989e+178+0j) 

Min value 1 = (4.906455409488344e+178+0j)
Min value 2 = (4.906455409488339e+178+0j) 

Avg diff = (3.944647615055475e+163+0j)
Std diff = inf
Max diff = (1.1517219314030583e+164+0j)
Min diff = 0j 

Mat 1 [3,4] = [5.28155723e+178+0.j]
Mat 2 [3,4] = (5.281557225576491e+178+0j)
Ex. diff [3,4] = (3.599131035634557e+163+0j)


In [134]:
print("TRr:\n")
diff_matrix_2d(Mx,My, TRr1, TRr2)

TRr:

Max value 1 = (6.211068229295988e+178+0j)
Max value 2 = (6.2110682292959894e+178+0j) 

Min value 1 = (4.906455409488341e+178+0j)
Min value 2 = (4.9064554094883395e+178+0j) 

Avg diff = (2.5265899870154593e+163+0j)
Std diff = inf
Max diff = (9.357740692649848e+163+0j)
Min diff = 0j 

Mat 1 [3,4] = [5.07039815e+178+0.j]
Mat 2 [3,4] = (5.0703981497851805e+178+0j)
Ex. diff [3,4] = (7.198262071269114e+162+0j)


In [135]:
print("TUr:\n")
diff_matrix_2d(Mx,My, TUr1, TUr2)

TUr:

Max value 1 = (5.278956369111059e+178+0j)
Max value 2 = (5.278956369111052e+178+0j) 

Min value 1 = (4.28227687274609e+178+0j)
Min value 2 = (4.28227687274609e+178+0j) 

Avg diff = (2.281849076592309e+163+0j)
Std diff = inf
Max diff = (7.198262071269114e+163+0j)
Min diff = 0j 

Mat 1 [3,4] = [4.42757646e+178+0.j]
Mat 2 [3,4] = (4.4275764649826666e+178+0j)
Ex. diff [3,4] = (7.198262071269114e+162+0j)


In [136]:
print("TDr:\n")
diff_matrix_2d(Mx,My, TDr1, TDr2)

TDr:

Max value 1 = (5.278956369111052e+178+0j)
Max value 2 = (5.278956369111052e+178+0j) 

Min value 1 = (4.2822768727460896e+178+0j)
Min value 2 = (4.28227687274609e+178+0j) 

Avg diff = (2.065901214454236e+163+0j)
Std diff = inf
Max diff = (6.478435864142203e+163+0j)
Min diff = 0j 

Mat 1 [3,4] = [4.43076438e+178+0.j]
Mat 2 [3,4] = (4.4307643779014256e+178+0j)
Ex. diff [3,4] = (1.439652414253823e+163+0j)


In [137]:
print("Difference of Hamiltonians: \n")
diff_matrix_3d(Mx, My, n, H_ele1, H_ele2)

Difference of Hamiltonians: 

Max value mat 1 = (-8.086310591251115e+179+0j)
Max value mat 2 = (-8.086310591251116e+179+0j) 

Min value mat 1 = (-1.939361639056334e+181+0j)
Min value mat 2 = (-1.9393616390563346e+181+0j) 

Avg diff = (3.644790511735376e+165+0j)
Std diff = inf
Max diff = (1.8427550902448932e+166+0j)
Min diff = 0j 

Mat 1 [3,4,5] = (-1.0104124419939489e+181+0j)
Mat 2 [3,4,5] = (-1.0104124419939485e+181+0j)
Ex. diff [3,4,5] = (3.6855101804897865e+165+0j)


Note: this errors are really small